In [387]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import time
import random

In [397]:
titles = []
years = []
durations = []
genres = []
imdb_ratings = []
votes =[]
gross = []

movie_list = soup.find_all(class_='lister-item mode-advanced')

In [398]:
for page in range(1,1001,50):
    
    headers = {'Accept-Language':'en-US,en,q=0.5'}
    results = requests.get(f'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&start={page}&ref_=adv_nxt',headers = headers)
    soup = bs(results.content)
    movie_list = soup.find_all(class_='lister-item mode-advanced')
    
    time.sleep(random.randint(3,7))
    
    for content in movie_list:

        title = content.h3.a.text
        titles.append(title)

        year = content.find(class_='lister-item-year text-muted unbold').text
        year = re.sub('[()]', '', year)
        if year.isdecimal() == True:
            years.append(int(year))
        else:
            years.append('-')

        duration = content.p.find('span',class_='runtime').text if content.p.find('span',class_='runtime').text else '-'
        duration = re.findall(r'\d+', duration)
        durations.append(int(duration[0]))

        genre = content.find(class_='genre').text
        genre = genre.strip()
        genres.append(genre)

        imdb_rating = content.find(class_='inline-block ratings-imdb-rating').text
        imdb_rating = float(re.sub('\n','',imdb_rating))
        imdb_ratings.append(imdb_rating)

        nv = content.find_all('span', attrs = {'name':'nv'})

        vote = nv[0].text
        vote = vote.replace(',','')
        votes.append(int(vote))

        if len(nv) > 1:
            gros = nv[1].text 
            gros  = re.findall(r'\d+', gros)
            gros = '.'.join(gros)
            gross.append(float(gros))
        else: 
            gross.append('-')

In [402]:
movies = pd.DataFrame({'Movie':titles, 'Year':years, 'Duration(min)':durations, 'Genre':genres,
                      'IMDb':imdb_ratings, 'Votes': votes, 'Gross (Mill)':gross})
movies

,Movie,Year,Duration(min),Genre,IMDb,Votes,Gross
0,The Shawshank Redemption,1994,142,Drama,9.3,2313251,28.34
1,The Godfather,1972,175,"Crime, Drama",9.2,1597049,134.97
2,The Dark Knight,2008,152,"Action, Crime, Drama",9.0,2275850,534.86
3,The Godfather: Part II,1974,202,"Crime, Drama",9.0,1115603,57.3
4,12 Angry Men,1957,96,"Crime, Drama",9.0,680406,4.36
...,...,...,...,...,...,...,...
995,Giant,1956,201,"Drama, Western",7.6,33710,-
996,From Here to Eternity,1953,118,"Drama, Romance, War",7.6,42975,30.5
997,Lifeboat,1944,97,"Drama, War",7.6,26225,-
998,Snow White and the Seven Dwarfs,1937,83,"Animation, Family, Fantasy",7.6,179662,184.93


In [400]:
movies.to_csv('movies_imdb1.csv', encoding = 'utf_8')